# Product Quantization Cluster Size Analysis

This notebook analyzes how cluster sizes vary when applying PQ clustering to different data distributions.

In [1]:
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from collections import Counter

## Configuration

In [2]:
# PQ parameters
n_points = 10000
dimension = 128
n_subvectors = 8  # Split dimension into subvectors
n_clusters = 256  # Number of clusters per subvector (typically 256 for 8-bit codes)

# Ensure dimension is divisible by n_subvectors
subvector_dim = dimension // n_subvectors
print(f"Dimension: {dimension}, Subvectors: {n_subvectors}, Subvector dim: {subvector_dim}")
print(f"Clusters per subvector: {n_clusters}")

np.random.seed(42)

Dimension: 128, Subvectors: 8, Subvector dim: 16
Clusters per subvector: 256


## Generate Different Data Distributions

In [3]:
def generate_distributions(n, d):
    """Generate various data distributions."""
    distributions = {}
    
    # 1. Uniform distribution [0, 1]
    distributions['Uniform [0,1]'] = np.random.uniform(0, 1, (n, d))
    
    # 2. Standard Gaussian
    distributions['Gaussian (0,1)'] = np.random.randn(n, d)
    
    # 3. Wide Gaussian
    distributions['Gaussian (0,5)'] = np.random.randn(n, d) * 5
    
    # 4. Mixture of Gaussians (10 components)
    n_mixture = 10
    cluster_centers = np.random.randn(n_mixture, d) * 5
    cluster_assignments = np.random.randint(0, n_mixture, n)
    mixture = cluster_centers[cluster_assignments] + np.random.randn(n, d) * 1.0
    distributions['Mixture of Gaussians (10)'] = mixture
    
    # 5. Sparse data (mostly zeros)
    sparse = np.random.randn(n, d) * 0.1
    mask = np.random.rand(n, d) > 0.9  # Only 10% non-zero
    distributions['Sparse (10% density)'] = sparse * mask
    
    # 6. Heavy-tailed (Laplace)
    distributions['Laplace (0,1)'] = np.random.laplace(0, 1, (n, d))
    
    # 7. Zipf distribution (power-law)
    # Generate Zipf values and map to vectors
    zipf_values = np.random.zipf(1.5, (n, d))
    # Normalize to reasonable range
    zipf_data = (zipf_values - zipf_values.mean()) / (zipf_values.std() + 1e-8)
    distributions['Zipf (a=1.5)'] = zipf_data
    
    return distributions

distributions = generate_distributions(n_points, dimension)
print(f"Generated {len(distributions)} distributions")

Generated 7 distributions


## Apply Product Quantization

In [4]:
def apply_pq_clustering(data, n_subvectors, n_clusters):
    """
    Apply Product Quantization to data.
    
    Returns cluster size statistics for each subvector.
    """
    n_samples, dim = data.shape
    subvector_dim = dim // n_subvectors
    
    cluster_size_stats = []
    
    for i in range(n_subvectors):
        # Extract subvector
        start_idx = i * subvector_dim
        end_idx = (i + 1) * subvector_dim
        subvector_data = data[:, start_idx:end_idx]
        
        # Cluster this subvector
        kmeans = KMeans(n_clusters=n_clusters, random_state=42, n_init=10)
        labels = kmeans.fit_predict(subvector_data)
        
        # Count cluster sizes
        cluster_counts = Counter(labels)
        sizes = list(cluster_counts.values())
        
        cluster_size_stats.append({
            'subvector': i,
            'sizes': sizes,
            'mean': np.mean(sizes),
            'std': np.std(sizes),
            'min': np.min(sizes),
            'max': np.max(sizes),
            'empty_clusters': n_clusters - len(cluster_counts)
        })
    
    return cluster_size_stats

## Analyze Cluster Sizes for Each Distribution

In [5]:
results = {}

for dist_name, data in distributions.items():
    print(f"\n{dist_name}:")
    print("-" * 50)
    
    stats = apply_pq_clustering(data, n_subvectors, n_clusters)
    results[dist_name] = stats
    
    # Aggregate statistics across all subvectors
    all_means = [s['mean'] for s in stats]
    all_stds = [s['std'] for s in stats]
    total_empty = sum(s['empty_clusters'] for s in stats)
    
    print(f"  Mean cluster size: {np.mean(all_means):.2f} ± {np.std(all_means):.2f}")
    print(f"  Std of cluster sizes: {np.mean(all_stds):.2f} ± {np.std(all_stds):.2f}")
    print(f"  Total empty clusters: {total_empty}/{n_subvectors * n_clusters}")


Uniform [0,1]:
--------------------------------------------------
  Mean cluster size: 39.06 ± 0.00
  Std of cluster sizes: 5.82 ± 0.27
  Total empty clusters: 0/2048

Gaussian (0,1):
--------------------------------------------------
  Mean cluster size: 39.06 ± 0.00
  Std of cluster sizes: 6.62 ± 0.25
  Total empty clusters: 0/2048

Gaussian (0,5):
--------------------------------------------------
  Mean cluster size: 39.06 ± 0.00
  Std of cluster sizes: 6.46 ± 0.21
  Total empty clusters: 0/2048

Mixture of Gaussians (10):
--------------------------------------------------
  Mean cluster size: 39.06 ± 0.00
  Std of cluster sizes: 8.10 ± 0.59
  Total empty clusters: 0/2048

Sparse (10% density):
--------------------------------------------------
  Mean cluster size: 39.06 ± 0.00
  Std of cluster sizes: 176.97 ± 3.15
  Total empty clusters: 0/2048

Laplace (0,1):
--------------------------------------------------
  Mean cluster size: 39.06 ± 0.00
  Std of cluster sizes: 12.54 ± 0.35

## Summary Statistics

In [6]:
# Create summary table
summary_data = []

for dist_name, stats in results.items():
    all_sizes = []
    for subvec_stats in stats:
        all_sizes.extend(subvec_stats['sizes'])
    
    total_empty = sum(s['empty_clusters'] for s in stats)
    
    summary_data.append({
        'Distribution': dist_name,
        'Mean Size': f"{np.mean(all_sizes):.2f}",
        'Std Size': f"{np.std(all_sizes):.2f}",
        'Min Size': f"{np.min(all_sizes):.0f}",
        'Max Size': f"{np.max(all_sizes):.0f}",
        'Empty Clusters': f"{total_empty}/{n_subvectors * n_clusters}"
    })

# Display as formatted table
print("\n" + "=" * 100)
print("SUMMARY: Cluster Size Statistics by Distribution")
print("=" * 100)
print(f"{'Distribution':<25} {'Mean Size':<12} {'Std Size':<12} {'Min Size':<10} {'Max Size':<10} {'Empty Clusters':<15}")
print("-" * 100)

for row in summary_data:
    print(f"{row['Distribution']:<25} {row['Mean Size']:<12} {row['Std Size']:<12} "
          f"{row['Min Size']:<10} {row['Max Size']:<10} {row['Empty Clusters']:<15}")


SUMMARY: Cluster Size Statistics by Distribution
Distribution              Mean Size    Std Size     Min Size   Max Size   Empty Clusters 
----------------------------------------------------------------------------------------------------
Uniform [0,1]             39.06        5.83         22         62         0/2048         
Gaussian (0,1)            39.06        6.63         14         65         0/2048         
Gaussian (0,5)            39.06        6.46         12         70         0/2048         
Mixture of Gaussians (10) 39.06        8.12         2          76         0/2048         
Sparse (10% density)      39.06        176.99       1          2914       0/2048         
Laplace (0,1)             39.06        12.55        1          90         0/2048         
Zipf (a=1.5)              39.06        539.56       1          8858       0/2048         
